In [1]:
from datasets import Dataset, DatasetDict
import pandas as pd
import os

# Finetune on imbalanced dataset

In [2]:
train = pd.read_csv(os.path.join("data-imbalance", "train-imbalance.csv"))   
val = pd.read_csv(os.path.join("data-imbalance", "dev-imbalance.csv"))

In [3]:
train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)

In [4]:
dataset_dict = DatasetDict({'train': train_dataset, 'validation': val_dataset})

In [5]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 5868
    })
    validation: Dataset({
        features: ['tweet', 'label'],
        num_rows: 1966
    })
})

In [6]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/Users/dimitris/miniconda3/envs/text/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [8]:
def preprocess_function(examples):
    return tokenizer(examples["tweet"], truncation=True)

In [9]:
tokenized_twitter = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/5868 [00:00<?, ? examples/s]

Map:   0%|          | 0/1966 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

/Users/dimitris/miniconda3/envs/text/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [12]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:

training_args = TrainingArguments(
    output_dir="semantic-bert-imbalanced-dataset",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_twitter["train"],
    eval_dataset=tokenized_twitter["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.push_to_hub()

  0%|          | 0/7340 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/123 [00:00<?, ?it/s]

{'eval_loss': 0.823654294013977, 'eval_accuracy': 0.6017293997965412, 'eval_runtime': 4.9998, 'eval_samples_per_second': 393.215, 'eval_steps_per_second': 24.601, 'epoch': 1.0}
{'loss': 0.7608, 'learning_rate': 1.8637602179836514e-05, 'epoch': 1.36}


  0%|          | 0/123 [00:00<?, ?it/s]

...

{'eval_loss': 3.404203414916992, 'eval_accuracy': 0.5930824008138352, 'eval_runtime': 3.9481, 'eval_samples_per_second': 497.961, 'eval_steps_per_second': 31.154, 'epoch': 19.0}
{'loss': 0.0078, 'learning_rate': 9.264305177111717e-07, 'epoch': 19.07}


  0%|          | 0/123 [00:00<?, ?it/s]

{'eval_loss': 3.3940141201019287, 'eval_accuracy': 0.5925737538148524, 'eval_runtime': 4.054, 'eval_samples_per_second': 484.954, 'eval_steps_per_second': 30.34, 'epoch': 20.0}
{'train_runtime': 1236.5448, 'train_samples_per_second': 94.91, 'train_steps_per_second': 5.936, 'train_loss': 0.13999554968010178, 'epoch': 20.0}


events.out.tfevents.1704278749.Dimitriss-MacBook-Pro.local.3889.0:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/J1mb0o/semantic-bert-imbalanced-dataset/commit/82679964d3339e08595b5d33dc878eb80d105584', commit_message='End of training', commit_description='', oid='82679964d3339e08595b5d33dc878eb80d105584', pr_url=None, pr_revision=None, pr_num=None)

# Finetuning on balanced dataset

In [16]:
train = pd.read_csv(os.path.join("data-balance", "train-balance.csv"))   
val = pd.read_csv(os.path.join("data-balance", "dev-balance.csv"))

In [17]:
train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(val)

In [18]:
dataset_dict = DatasetDict({'train': train_dataset, 'validation': val_dataset})

In [19]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 2550
    })
    validation: Dataset({
        features: ['tweet', 'label'],
        num_rows: 1173
    })
})

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [21]:
def preprocess_function(examples):
    return tokenizer(examples["tweet"], truncation=True)

In [22]:
tokenized_twitter = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/2550 [00:00<?, ? examples/s]

Map:   0%|          | 0/1173 [00:00<?, ? examples/s]

In [23]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [24]:
import evaluate

accuracy = evaluate.load("accuracy")

In [25]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [26]:
id2label = {0: "negative", 1: "neutral", 2: "positive"}
label2id = {"negative": 0, "neutral": 1, "positive": 2}

In [27]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:

training_args = TrainingArguments(
    output_dir="semantic-bert-balanced-dataset",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_twitter["train"],
    eval_dataset=tokenized_twitter["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.push_to_hub()

  0%|          | 0/3200 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/74 [00:00<?, ?it/s]

{'eval_loss': 0.8837119340896606, 'eval_accuracy': 0.5745950554134698, 'eval_runtime': 2.5222, 'eval_samples_per_second': 465.069, 'eval_steps_per_second': 29.339, 'epoch': 1.0}


  0%|          | 0/74 [00:00<?, ?it/s]

...

{'eval_loss': 3.179137945175171, 'eval_accuracy': 0.5473145780051151, 'eval_runtime': 2.1548, 'eval_samples_per_second': 544.356, 'eval_steps_per_second': 34.341, 'epoch': 19.0}


  0%|          | 0/74 [00:00<?, ?it/s]

{'eval_loss': 3.1861555576324463, 'eval_accuracy': 0.5447570332480819, 'eval_runtime': 2.1511, 'eval_samples_per_second': 545.31, 'eval_steps_per_second': 34.401, 'epoch': 20.0}
{'train_runtime': 566.6488, 'train_samples_per_second': 90.003, 'train_steps_per_second': 5.647, 'train_loss': 0.1629483714699745, 'epoch': 20.0}


events.out.tfevents.1704280023.Dimitriss-MacBook-Pro.local.3889.1:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/J1mb0o/semantic-bert-balanced-dataset/commit/34a05ece7a91c7dc05d35cb6e14bb7e2c323179f', commit_message='End of training', commit_description='', oid='34a05ece7a91c7dc05d35cb6e14bb7e2c323179f', pr_url=None, pr_revision=None, pr_num=None)

# Random Forest

In [83]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [84]:
train_dataset = pd.read_csv(os.path.join("data-imbalance", "train-imbalance.csv"))

In [85]:
X_train, y_train = train_dataset["tweet"], train_dataset["label"]

In [88]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=200, random_state=42)),
])

In [89]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 RandomForestClassifier(n_estimators=200, random_state=42))])

# Eval random forest

In [103]:
import evaluate
def model_evaluation(predicted_class=None, true_class=None):
    if predicted_class is None or true_class is None:
        raise ValueError("predicted and true must be not None")
    
    accuracy = evaluate.load("accuracy")
    acc = accuracy.compute(predictions=predicted_class, references=true_class, )
    
    precision = evaluate.load("precision")
    prec = precision.compute(predictions=predicted_class, references=true_class, average=None)
    prec_weigted = precision.compute(predictions=predicted_class, references=true_class, average="macro")

    recall = evaluate.load("recall")
    rec = recall.compute(predictions=predicted_class, references=true_class, average=None)
    rec_weigted = recall.compute(predictions=predicted_class, references=true_class, average="macro")

    f1 = evaluate.load("f1")
    calc_f1 = f1.compute(predictions=predicted_class, references=true_class, average=None)
    f1_weighted = f1.compute(predictions=predicted_class, references=true_class, average="macro")
    return acc, prec, prec_weigted, rec, rec_weigted, calc_f1, f1_weighted
    # return acc

## Unbalanced test set

In [104]:
test_set_unb = pd.read_csv(os.path.join("data-imbalance", "test-imbalance.csv"))
X_test_unb, y_test_unb = test_set_unb["tweet"], test_set_unb["label"]

In [105]:
predicted_unb = text_clf.predict(X_test_unb)

In [106]:
unbalanced_eval = model_evaluation(predicted_unb, y_test_unb)

In [111]:
unbalanced_eval

({'accuracy': 0.5215},
 {'precision': array([0.66666667, 0.45493562, 0.53004535])},
 {'precision': 0.550549213486062},
 {'recall': array([0.00615385, 0.15565345, 0.94064386])},
 {'recall': 0.36748372004685215},
 {'f1': array([0.01219512, 0.23194748, 0.67802756])},
 {'f1': 0.3073900539133287})

In [110]:
import pandas as pd

## Balanced

In [107]:
test_set_bal = pd.read_csv(os.path.join("data-balance", "test-balance.csv"))
X_test_bal, y_test_bal = test_set_bal["tweet"], test_set_bal["label"]

In [108]:
balanced_predicted = text_clf.predict(X_test_bal)

In [109]:
balanced_eval = model_evaluation(balanced_predicted, y_test_bal)

## Export results to a dataframe

In [112]:
resutls_df = pd.DataFrame(columns=["Model", 
                      "Accuracy", 
                      "Pre_{Negative}", 
                      "Pre_{Neu}", 
                      "Pre_{Pos}", 
                      "Pre_{Weighted}", 
                      "Rec_{Negative}", 
                      "Rec_{Neu}", 
                      "Rec_{Pos}", 
                      "Rec_{Weighted}", 
                      "F1_{Negative}", 
                      "F1_{Neu}", 
                      "F1_{Pos}", 
                      "F1_{Weighted}"],
                      data=[["Random Forest (unbalanced)", 
                              unbalanced_eval[0]["accuracy"], 
                              unbalanced_eval[1]["precision"][0], 
                              unbalanced_eval[1]["precision"][1],
                              unbalanced_eval[1]["precision"][2],
                              unbalanced_eval[2]["precision"],
                              unbalanced_eval[3]["recall"][0],
                              unbalanced_eval[3]["recall"][1],
                              unbalanced_eval[3]["recall"][2],
                              unbalanced_eval[4]["recall"], 
                              unbalanced_eval[5]["f1"][0],
                              unbalanced_eval[5]["f1"][1],
                              unbalanced_eval[5]["f1"][2],
                              unbalanced_eval[6]["f1"]],
                              ["Random Forest (balanced)",
                              balanced_eval[0]["accuracy"],
                              balanced_eval[1]["precision"][0],
                              balanced_eval[1]["precision"][1],
                              balanced_eval[1]["precision"][2],
                              balanced_eval[2]["precision"],
                              balanced_eval[3]["recall"][0],
                              balanced_eval[3]["recall"][1],
                              balanced_eval[3]["recall"][2],
                              balanced_eval[4]["recall"],
                              balanced_eval[5]["f1"][0],
                              balanced_eval[5]["f1"][1],
                              balanced_eval[5]["f1"][2],
                              balanced_eval[6]["f1"]],
                                                  
                        ]
                      )

In [115]:
resutls_df.round(4)

,Model,Accuracy,Pre_{Negative},Pre_{Neu},Pre_{Pos},Pre_{Weighted},Rec_{Negative},Rec_{Neu},Rec_{Pos},Rec_{Weighted},F1_{Negative},F1_{Neu},F1_{Pos},F1_{Weighted}
0,Random Forest (unbalanced),0.5215,0.6667,0.4549,0.5300,0.5505,0.0062,0.1557,0.9406,0.3675,0.0122,0.2319,0.678,0.3074
1,Random Forest (balanced),0.3631,0.6667,0.3605,0.3624,0.4632,0.0062,0.1631,0.9200,0.3631,0.0122,0.2246,0.520,0.2523


In [114]:
resutls_df.to_latex("random_forest.tex", index=False, float_format="%.4f")